# Creating spatial variables

First thing I will try here is to calculate the average snow off date by watershed for catchment outlet points.

* Resample modis raster from 500 m to 5 m so that the weighted flow accumulation correctly sums down the network.
* Use weighted flow accumulation to calculate across a flow direction grid. We'll need to loop through the different flow direction grids and also the different years of modis snow metrics tifs as weights.
* Use an inner buffer on each catchment and extract raster cell with largest flow accumulation.
* With these "catchment outlet cells", extract the mean modis date of snow off for each catchment.

In [12]:
import arcpy
import os 
import time
import datetime

print(dir())

['In', 'Out', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i10', '_i11', '_i12', '_i2', '_i3', '_i4', '_i5', '_i6', '_i7', '_i8', '_i9', '_ih', '_ii', '_iii', '_oh', '_pydevd_bundle', 'akssf_sa', 'arcpy', 'datetime', 'exit', 'fdr_raster', 'get_ipython', 'lcld', 'math', 'modis_raster', 'modis_sa', 'numpy', 'os', 'pydev_jupyter_vars', 'quit', 'sys', 'tempdir', 'time']


Set up temporary directory and snap raster and other settings for test processing area - 19020103.

In [10]:
modis_raster = "W:/GIS/MODIS_Snow_Metrics/2019_snowyear_metrics_v006.tif"
fdr_raster = "T:/Aquatic/AKSSF/NHDPlus/CopperRiver_20210408/extracts/HRNHDPlusRasters19020103/fdr.tif"
akssf_sa = "T:/Aquatic/AKSSF/AKSSF_Hydrography.gdb/AKSSF_studyarea_HUC8"

tempdir = "W:\GIS\AKSSF Southcentral\Temporary"
arcpy.env.workspace = tempdir
arcpy.env.snapRaster = fdr_raster
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

Test out for one fdr first and then try in a for statement. Dustin provided code for looping through gdb in a workspace in the catchment_processing python script.

lcld is acronym for last day of the longest continuous snow season, band 5 in each tif.

* first sum the lcld values using the fdr
* divide by the fac to get the mean lcld by all upstream cells
* create a stream raster
* extract the mean wtd lcld for just the stream cells

In [6]:
modis_sa = arcpy.sa.ExtractByMask(modis_raster, akssf_sa)

In [11]:
lcld = arcpy.ia.ExtractBand(modis_sa, [5])
lcld.save(os.path.join(tempdir, "lcld_sa_2019.tif"))


STOPPED HERE THE CODE BELOW THROWS AN ERROR, CAN'T GET RESAMPLE TO WORK.

In [16]:
start = datetime.datetime.now()
lcld_5m = arcpy.management.Resample(lcld, "", 5, "Bilinear")
stop = datetime.datetime.now()
elapsed = stop - start

print ("Process complete ", elapsed)

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000735: Output Raster Dataset: Value is required
Failed to execute (Resample).


In [14]:
lcld_5m.save(os.path.join(tempdir, "lcld_5m_2019.tif")

SyntaxError: unexpected EOF while parsing (<ipython-input-14-d28bcc272381>, line 2)

In [ ]:


tic = time.perf_counter()


lcld_sum_19020103_2019 = arcpy.sa.FlowAccumulation(fdr_raster, lcld)
arcpy.sa.

toc = time.perf_counter()
print(f"Summed lcld across one HUC8 in {toc - tic:0.4f} seconds")

In [7]:
fac_raster = "T:/Aquatic/AKSSF/NHDPlus/CopperRiver_20210408/extracts/HRNHDPlusRasters19020103/fac.tif"
lcld_wtdmn_19020103_2019 = lcld_sum_19020103_2019/fac_raster


In [10]:
#surface water raster that comes with NHDPlus
sw_raster = "T:/Aquatic/AKSSF/NHDPlus/CopperRiver_20210408/extracts/HRNHDPlusRasters19020103/swnet.tif"

#print raster attribute table
for r in arcpy.SearchCursor(sw_raster, "", "", "VALUE; COUNT"):
    v = r.getValue("VALUE")
    c = r.getValue("COUNT")
    print (v, c)

remap_tbl = arcpy.sa.RemapValue([[1, 1], [2, "NoData"]])
    
#just extract streams, value == 1
str_raster = arcpy.sa.Reclassify(sw_raster, "VALUE", remap_tbl)


1 3753915.0
2 3635347.0


In [11]:
lcld_str_19020103_2019 = arcpy.sa.ExtractByMask(lcld_wtdmn_19020103_2019, str_raster)

In [ ]:
str_raster.save("W:/GIS/AKSSF Southcentral/AKSSF Southcentral.gdb/str_19020103")